# Homework10 Equality constrained minimization

In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat

## Maximum likelihood prediction of team abilities
$n$: number of teams, $m$: number of matches/games  
The winning result of $j$ in its match with $k$ is defined as:
$$\textbf{prob}(a_j-a_k+v>0) \quad v\sim \mathcal{N}(0,\sigma^2)$$
$$\textbf{prob}(a_j-a_k+v>0)=\textbf{prob}(v>a_k-a_j)= \textbf{prob}(v<a_j-a_k)=\varPhi(\frac{a_j-a_k}{\sigma})$$

The problem is set as find estimation of abilities $a$ to maximize the likelihood of all games given their results.
$$\text{maximize}\quad \sum_{i=1}^m log \varPhi(\frac{a_j^{(i)}-a_k^{(i)}}{\sigma})$$
$$\text{subject to}\quad 0 \leq a \leq 1$$
concave function with affine inequalities

In [2]:
data = loadmat("team_data.mat")
A = data['A']
m, n = data['m'][0, 0], data['n'][0, 0]
sigma = data['sigma'][0, 0]
test = data['test']
train = data['train']

In [3]:
a = cp.Variable(n)
cst = [0 <= a, a <= 1]
mu = A @ a
obj = cp.Maximize(cp.sum(cp.log_normcdf(mu / sigma)))
prob = cp.Problem(obj, cst)

prob.solve()
print(a.value)

[ 1.00000000e+00 -3.07024986e-21  6.78273774e-01  3.68704618e-01
  7.90021663e-01  5.81306405e-01  3.87382448e-01  8.54414113e-02
  6.70046253e-01  5.77486181e-01]


In [4]:
len = test.shape[0]
test_res = np.zeros((len, ))
for i in range(len):
    j, k = test[i, :2]
    res = test[i, 2]
    prediction = np.sign(a.value[j - 1] - a.value[k - 1])
    test_res[i] = res == prediction
print("Accuracy of ML: \t %.3f" % (np.sum(test_res) / test_res.shape[0]))

Accuracy of ML: 	 0.867


In [5]:
# simpler method
test_res = np.zeros((len, ))
for i in range(len):
    test_res[i] = test[i, 2] == train[i, 2]
print("Accuracy of simpler method: \t %.3f" % (np.sum(test_res) / test_res.shape[0]))

Accuracy of simpler method: 	 0.756


## Allocation of interdiction effort


The original problem is mini-max of the probability of paths from node 1 to node n:
$$
\text{minimize} \quad z\\
\text{subject to} \quad \forall path, \prod_{j=1,j\in \text{ path }}^m p_j \leq z
$$

It can be converted to:
$$\text{maximize} \quad z' = -log(z)\\
\text{subject to} \quad \forall path, -\sum_{j=1, j\in \text{ path }}^m logp_j = \sum_{j=1, j\in \text{ path }}^m a_jx_j \geq z'
$$
Actually, $z'$ is the shortest path in the graph given $x$. We want to maximize the length of shortest path.

### LP
The arriving probability of node i is denoted as $z_i$. Similarly, $z_i'=-logz_i$.  
The shortest path from source to every node is an extension to the shortest path of its one predecessor.

Therefore, we have $\forall j \in \{\text{pred}_i\}, z_i' \leq z_j' + a_{ij}x_{ij} \equiv z_i'-z_j' \leq a_{ij}x_{ij}$.
It is equivalent to $$A^T z' \leq a\cdot x$$

The problem goes to LP form:
$$\text{maximize} \quad z_n'\\
\text{subject to} \quad A^T z' \leq \text{diag}(a) x $$
with other constraints:
1. $0 \leq x \leq x_{max}$
2. $1^T x \leq B$

In [6]:
data = loadmat("interdict_data.mat")
a = data['a']
A = data['A']
edges = data['edges']
x_max = data['x_max']
B, j, m, n = data['B'][0, 0], data['j'][0, 0], data['m'][0, 0], data['n'][0, 0]

In [7]:
x = cp.Variable((m, 1), pos=True)
z = cp.Variable((n, 1), pos=True)

cst = [A.T @ z <= cp.multiply(a, x), x <= x_max, cp.sum(x) <= B, z[0] == 0]
obj = cp.Maximize(z[n - 1])
prob = cp.Problem(obj, cst)

p = prob.solve()
# convert from z' to z
p = np.exp(-p)
print("Mini-max: \t %.3f" % p)

Mini-max: 	 0.043


In [8]:
# uniform
x = np.ones((m, 1)) * B / m
prob = cp.Problem(obj, [A.T @ z <= cp.multiply(a, x), z[0] == 0])
p = prob.solve()
p = np.exp(-p)
print("Uniform: \t %.3f" % p)

Uniform: 	 0.247
